In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import logging
import os
from datetime import datetime
import traceback
import pandas as pd

# Import our modules
#from pdf_processor import FinanceBenchProcessor
#from data_processor import FinancialDataProcessor
#from retrieval_system import AdvancedRetrievalSystem
#from qa_system import FinancialQASystem
#from sentiment_analyzer import FinancialSentimentAnalyzer
#from anomaly_detector import FinancialAnomalyDetector


%run pdf_processor.ipynb
%run data_processor.ipynb
%run retrieval_system.ipynb
%run qa_system.ipynb
%run sentiment_analyzer.ipynb
%run anomaly_detector.ipynb


# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Initialize systems
fb_processor = None
processor = FinancialDataProcessor()
retrieval_system = AdvancedRetrievalSystem()
qa_system = FinancialQASystem()
sentiment_analyzer = FinancialSentimentAnalyzer()
anomaly_detector = FinancialAnomalyDetector()

# Global variables
documents_loaded = False
questions_loaded = False
all_documents = {}
all_questions = []

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint."""
    return jsonify({
        'status': 'healthy',
        'timestamp': datetime.now().isoformat(),
        'documents_loaded': documents_loaded,
        'questions_loaded': questions_loaded
    })

@app.route('/initialize', methods=['POST'])
def initialize_system():
    """Initialize the system with FinanceBench data."""
    global fb_processor, documents_loaded, questions_loaded, all_documents, all_questions
    
    try:
        data = request.json
        pdfs_dir = data.get('pdfs_dir', 'pdfs')
        data_dir = data.get('data_dir', 'data')
        
        # Initialize processor
        fb_processor = FinanceBenchProcessor(pdfs_dir, data_dir)
        
        # Load all documents
        logger.info("Loading PDF documents...")
        all_documents = fb_processor.load_all_documents()
        
        # Load all questions
        logger.info("Loading questions...")
        all_questions = fb_processor.get_all_questions()
        
        # Process documents for retrieval
        logger.info("Processing documents for retrieval...")
        all_chunks = []
        
        for doc_name, doc_data in all_documents.items():
            chunks = processor.intelligent_chunking(doc_data['text'])
            
            # Add metadata to chunks
            for chunk in chunks:
                chunk['document_id'] = doc_name
                chunk['document_metadata'] = {
                    'doc_name': doc_name,
                    'company': doc_data['company'],
                    'doc_type': doc_data['doc_type'],
                    'doc_period': doc_data['doc_period'],
                    'gics_sector': doc_data['gics_sector']
                }
            
            all_chunks.extend(chunks)
        
        # Build retrieval index
        logger.info("Building retrieval index...")
        retrieval_system.build_index(all_chunks)
        
        documents_loaded = True
        questions_loaded = True
        
        return jsonify({
            'message': 'FinanceBench system initialized successfully',
            'total_documents': len(all_documents),
            'total_questions': len(all_questions),
            'total_chunks': len(all_chunks),
            'companies': list(set(doc['company'] for doc in all_documents.values()))
        })
        
    except Exception as e:
        logger.error(f"Error initializing system: {e}")
        logger.error(traceback.format_exc())
        return jsonify({'error': str(e)}), 500

@app.route('/ask', methods=['POST'])
def ask_question():
    """Answer questions using the loaded documents with debug output."""
    if not documents_loaded:
        return jsonify({'error': 'System not initialized. Call /initialize first.'}), 400
    
    try:
        data = request.json
        question = data.get('question', '')
        company_filter = data.get('company', None)
        doc_type_filter = data.get('doc_type', None)
        top_k = data.get('top_k', 5)
        
        if not question:
            return jsonify({'error': 'No question provided'}), 400
        
        print(f"\n🌟 QUESTION: {question}")
        print(f"🏢 Company filter: {company_filter}")
        print(f"📄 Doc type filter: {doc_type_filter}")
        
        # Retrieve relevant contexts
        contexts = retrieval_system.search(question, top_k=top_k)
        print(f"Retrieved {len(contexts)} initial chunks")
        
        # Filter contexts if filters provided
        if company_filter or doc_type_filter:
            filtered_contexts = []
            for context in contexts:
                metadata = context.get('document_metadata', {})
                if company_filter and metadata.get('company') != company_filter:
                    continue
                if doc_type_filter and metadata.get('doc_type') != doc_type_filter:
                    continue
                filtered_contexts.append(context)
            contexts = filtered_contexts
            print(f"After filtering: {len(contexts)} chunks remain")
        
        print(f"Final retrieved chunks:")
        for idx, chunk in enumerate(contexts):
            print(f"\n--- Chunk {idx+1} ---")
            # Show chunk text preview
            chunk_text = chunk.get('text', chunk.get('content', ''))
            print(f"Text preview: {chunk_text[:500]}")
            # Show metadata if available
            metadata = chunk.get('document_metadata', {})
            if metadata:
                print(f"Metadata: Company={metadata.get('company')}, Doc={metadata.get('doc_name')}")
        
        # Prepare context for QA (handle both formats)
        if contexts and isinstance(contexts[0], dict):
            if 'text' in contexts[0]:
                context_text = "\n\n".join([chunk['text'] for chunk in contexts])
            else:
                context_text = "\n\n".join([chunk.get('content', '') for chunk in contexts])
        else:
            context_text = ""
        
        print(f"\nCombined context length: {len(context_text)} characters")
        print(f"Context preview (first 300 chars): {context_text[:300]}...")
        
        if not context_text.strip():
            print("⚠️ WARNING: Empty context for QA!")
            return jsonify({
                'question': question,
                'answer': 'No relevant information found',
                'confidence': 0.0,
                'retrieved_contexts': len(contexts),
                'error': 'No relevant context retrieved'
            })
        
        # Generate answer
        answer_result = qa_system.answer_question(question, context_text)
        print(f"QA Result: {answer_result}")
        
        return jsonify({
            'question': question,
            'answer': answer_result.get('answer', ''),
            'confidence': answer_result.get('confidence', 0),
            'qa_confidence': answer_result.get('qa_confidence', 0),
            'similarity_confidence': answer_result.get('similarity_confidence', 0),
            'combined_confidence': answer_result.get('combined_confidence', 0),
            'source_chunks': answer_result.get('source_chunks', []),
            'context_preview': context_text[:200] + "..." if len(context_text) > 200 else context_text,
            'retrieved_contexts': len(contexts),
            'filters_applied': {
                'company': company_filter,
                'doc_type': doc_type_filter
            },
            'timestamp': datetime.now().isoformat()
        })
        
    except Exception as e:
        logger.error(f"Error in Q&A: {e}")
        logger.error(traceback.format_exc())
        print(f"❌ ERROR in /ask: {e}")
        return jsonify({'error': str(e)}), 500


@app.route('/evaluate_benchmarks', methods=['POST'])
def evaluate_benchmarks():
    """Evaluate system on FinanceBench questions."""
    if not documents_loaded or not questions_loaded:
        return jsonify({'error': 'System not initialized. Call /initialize first.'}), 400
    
    try:
        data = request.json
        max_questions = data.get('max_questions', 10)  # Limit for demo
        company_filter = data.get('company', None)
        question_type_filter = data.get('question_type', None)
        
        # Filter questions
        questions_to_evaluate = all_questions.copy()
        
        if company_filter:
            questions_to_evaluate = [q for q in questions_to_evaluate if q['company'] == company_filter]
        
        if question_type_filter:
            questions_to_evaluate = [q for q in questions_to_evaluate if q['question_type'] == question_type_filter]
        
        # Limit number of questions
        questions_to_evaluate = questions_to_evaluate[:max_questions]
        
        results = []
        correct_answers = 0
        
        for question_data in questions_to_evaluate:
            question = question_data['question']
            gold_answer = question_data['answer']
            
            # Get relevant contexts
            contexts = retrieval_system.search(question, top_k=5)
            
            # Filter contexts by document
            doc_name = question_data['doc_name']
            filtered_contexts = [
                ctx for ctx in contexts 
                if ctx.get('document_metadata', {}).get('doc_name') == doc_name
            ]
            
            # If no contexts from the specific document, use all contexts
            if not filtered_contexts:
                filtered_contexts = contexts
            
            # Generate answer
            answer_result = qa_system.answer_question(question, filtered_contexts)
            
            # Simple evaluation (you can make this more sophisticated)
            is_correct = _evaluate_answer(answer_result['answer'], gold_answer)
            if is_correct:
                correct_answers += 1
            
            results.append({
                'financebench_id': question_data['financebench_id'],
                'question': question,
                'predicted_answer': answer_result['answer'],
                'gold_answer': gold_answer,
                'is_correct': is_correct,
                'confidence': answer_result['confidence'],
                'company': question_data['company'],
                'question_type': question_data['question_type'],
                'doc_name': question_data['doc_name']
            })
        
        accuracy = correct_answers / len(results) if results else 0
        
        return jsonify({
            'evaluation_results': results,
            'summary': {
                'total_questions': len(results),
                'correct_answers': correct_answers,
                'accuracy': accuracy,
                'average_confidence': sum(r['confidence'] for r in results) / len(results) if results else 0
            },
            'filters_applied': {
                'company': company_filter,
                'question_type': question_type_filter,
                'max_questions': max_questions
            }
        })
        
    except Exception as e:
        logger.error(f"Error in benchmark evaluation: {e}")
        logger.error(traceback.format_exc())
        return jsonify({'error': str(e)}), 500

def _evaluate_answer(predicted: str, gold: str) -> bool:
    """Simple answer evaluation - you can make this more sophisticated."""
    # Remove common variations and normalize
    predicted = predicted.lower().strip()
    gold = gold.lower().strip()
    
    # Check if key numbers match
    import re
    pred_numbers = re.findall(r'[\d,]+\.?\d*', predicted)
    gold_numbers = re.findall(r'[\d,]+\.?\d*', gold)
    
    if pred_numbers and gold_numbers:
        # If both have numbers, check if any match
        return any(pnum in gold_numbers for pnum in pred_numbers)
    
    # Otherwise, check for substring match
    return gold in predicted or predicted in gold

@app.route('/company_analysis', methods=['POST'])
def company_analysis():
    """Perform comprehensive analysis for a specific company."""
    if not documents_loaded:
        return jsonify({'error': 'System not initialized. Call /initialize first.'}), 400
    
    try:
        data = request.json
        company = data.get('company', '')
        
        if not company:
            return jsonify({'error': 'No company specified'}), 400
        
        # Get company documents
        company_docs = {k: v for k, v in all_documents.items() if v['company'] == company}
        
        if not company_docs:
            return jsonify({'error': f'No documents found for company: {company}'}), 400
        
        # Get company questions
        company_questions = [q for q in all_questions if q['company'] == company]
        
        # Sentiment analysis
        sentiment_results = []
        for doc_name, doc_data in company_docs.items():
            chunks = processor.intelligent_chunking(doc_data['text'])
            doc_sentiment = sentiment_analyzer.analyze_chunks(chunks)
            aggregated_sentiment = sentiment_analyzer.aggregate_sentiment(doc_sentiment)
            
            sentiment_results.append({
                'doc_name': doc_name,
                'doc_type': doc_data['doc_type'],
                'doc_period': doc_data['doc_period'],
                'sentiment': aggregated_sentiment
            })
        
        # Extract financial metrics
        financial_metrics = []
        for doc_name, doc_data in company_docs.items():
            metrics = processor.extract_financial_metrics(doc_data['text'])
            if metrics:
                metrics['doc_name'] = doc_name
                metrics['doc_period'] = doc_data['doc_period']
                financial_metrics.append(metrics)
        
        # Anomaly detection
        anomaly_results = None
        if financial_metrics:
            anomaly_results = anomaly_detector.detect_financial_metric_anomalies(financial_metrics)
        
        return jsonify({
            'company': company,
            'documents_analyzed': len(company_docs),
            'questions_available': len(company_questions),
            'sentiment_analysis': sentiment_results,
            'financial_metrics': financial_metrics,
            'anomaly_detection': anomaly_results,
            'document_types': list(set(doc['doc_type'] for doc in company_docs.values())),
            'time_periods': sorted(list(set(doc['doc_period'] for doc in company_docs.values())))
        })
        
    except Exception as e:
        logger.error(f"Error in company analysis: {e}")
        logger.error(traceback.format_exc())
        return jsonify({'error': str(e)}), 500

@app.route('/list_companies', methods=['GET'])
def list_companies():
    """List all available companies."""
    if not documents_loaded:
        return jsonify({'error': 'System not initialized. Call /initialize first.'}), 400
    
    companies = {}
    for doc_name, doc_data in all_documents.items():
        company = doc_data['company']
        if company not in companies:
            companies[company] = {
                'company': company,
                'documents': 0,
                'doc_types': set(),
                'periods': set(),
                'sector': doc_data['gics_sector']
            }
        
        companies[company]['documents'] += 1
        companies[company]['doc_types'].add(doc_data['doc_type'])
        companies[company]['periods'].add(doc_data['doc_period'])
    
    # Convert sets to lists for JSON serialization
    for company_data in companies.values():
        company_data['doc_types'] = list(company_data['doc_types'])
        company_data['periods'] = sorted(list(company_data['periods']))
    
    return jsonify({
        'companies': list(companies.values()),
        'total_companies': len(companies)
    })

if __name__ == '__main__':
    app.run(debug=False, use_reloader=False, host='0.0.0.0', port=5000)



/Users/puchku-home/Downloads/Hackathon Project 2/.venv1/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
Device set to use cpu
Device set to use cpu


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.29.172:5000
INFO:werkzeug:Press CTRL+C to quit
